In [20]:


from google.colab import drive
drive.mount('/content/drive')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
img_rows, img_cols = 224, 224 #number of rows and columns to convert the images to
input_shape = (img_rows, img_cols, 3)#format to store the images (rows, columns,channels) called channels last


In [22]:
import numpy as np
import urllib.request
import cv2

In [23]:

def url_to_image(url):
	# download the image, convert it to a NumPy array, and then read
	# it into OpenCV format
	resp = urllib.request.urlopen(url)
	image = np.asarray(bytearray(resp.read()), dtype="uint8")
	image = cv2.imdecode(image, cv2.IMREAD_COLOR)
 
	# return the image
	return image


#%clear base_model


In [24]:
import tensorflow as tf

base_model = tf.keras.applications.ResNet50(input_shape=(224,224, 3),weights='imagenet', include_top = False)
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)

x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
preds = tf.keras.layers.Dense(6, activation ='softmax')(x)

model = tf.keras.models.Model(inputs=base_model.input, outputs=preds)

In [25]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

train_datagen = ImageDataGenerator()
valid_datagen = ImageDataGenerator()
    
train_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/Colab Notebooks/Train Data For selected 2 project/natural_images/N/Train',
        classes=['airplane','car','motorbike','person','flower', 'dog'],
        target_size=(img_rows, img_cols),#The target_size is the size of your input images,every image will be resized to this size
        batch_size=32,
        class_mode='categorical')

valid_generator = valid_datagen.flow_from_directory(
        '/content/drive/MyDrive/Colab Notebooks/Train Data For selected 2 project/natural_images/N/Validate',
        classes=['airplane','car','motorbike','person','flower', 'dog'],        
        target_size=(img_rows, img_cols),#The target_size is the size of your input images,every image will be resized to this size
        batch_size=32,
        class_mode='categorical')

Found 2298 images belonging to 6 classes.
Found 1063 images belonging to 6 classes.


In [26]:

for layer in base_model.layers:
  layer.trainable = False

In [27]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [28]:
history = model.fit(
      train_generator,
      steps_per_epoch=train_generator.n//train_generator.batch_size,
      epochs=5,
      validation_data=valid_generator,
      validation_steps=25)

Epoch 1/5
71/71 [==============================] - 443s 6s/step - loss: 0.1644 - accuracy: 0.9475 - val_loss: 0.0155 - val_accuracy: 0.9950
Epoch 2/5
71/71 [==============================] - 433s 6s/step - loss: 0.0586 - accuracy: 0.9872 - val_loss: 0.0132 - val_accuracy: 0.9950
Epoch 3/5
71/71 [==============================] - 433s 6s/step - loss: 0.0104 - accuracy: 0.9974 - val_loss: 0.0123 - val_accuracy: 0.9975
Epoch 4/5
71/71 [==============================] - 427s 6s/step - loss: 0.0078 - accuracy: 0.9987 - val_loss: 0.0123 - val_accuracy: 0.9962
Epoch 5/5
71/71 [==============================] - 426s 6s/step - loss: 3.1998e-04 - accuracy: 1.0000 - val_loss: 0.0161 - val_accuracy: 0.9987


In [29]:
model.save ( '/content/drive/My Drive/db/6C/Bipartite_Adam_pln_pp/model1/6C_Baseline/' )
# import tensorflow as tf
# model1 =tf.keras.models.load_model('/content/drive/My Drive/db/6C/Bipartite_Adam_pln_pp/model1/' )

INFO:tensorflow:Assets written to: /content/drive/My Drive/db/6C/Bipartite_Adam_pln_pp/model1/6C_Baseline/assets


In [17]:


import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
#test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input)



In [19]:
test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/Colab Notebooks/Train Data For selected 2 project/natural_images/N/Test',
        classes=['airplane','car','motorbike','person','flower', 'dog'], 
        target_size=(374, 374),#The target_size is the size of your input images,every image will be resized to this size
        batch_size=32,
        class_mode='categorical')

history=model.evaluate(
test_generator    
)

print(history)

Found 977 images belonging to 6 classes.
31/31 [==============================] - 382s 12s/step - loss: 0.2229 - accuracy: 0.9539
[0.22293275594711304, 0.9539406299591064]
